 # SQL Murder Mystery

 A crime has taken place and the detective needs your help. The detective gave
 you the crime scene report, but you somehow lost it.

 ## Clue 1

 You vaguely remember that the crime was a **murder** that occurred sometime on
 **Jan. 15, 2018** and that it took place in **SQL City**. Start by retrieving
 the corresponding crime scene report from the police department's database. If
 you want to get the most out of this mystery, try to work through it only
 using your SQL environment and refrain from using a notepad.

In [1]:
import pandas as pd
from IPython.display import Markdown, display


In [2]:
drivers_license = pd.read_csv("drivers_license.zip")
income = pd.read_csv("income.zip")
get_fit_now_members = pd.read_csv("get_fit_now_members.zip")
interview = pd.read_csv("interview.zip")
person = pd.read_csv("person.zip")
facebook_event_check_in = pd.read_csv("facebook_event_check_in.zip")
get_fit_now_check_in = pd.read_csv("get_fit_now_check_in.zip")
crime_scene_report = pd.read_csv("crime_scene_report.zip")


In [3]:
db = [
    drivers_license,
    income,
    get_fit_now_members,
    interview,
    person,
    facebook_event_check_in,
    get_fit_now_check_in,
    crime_scene_report,
]


 ![image.png](https://mystery.knightlab.com/schema.png)

In [4]:
deposition = (
    crime_scene_report.assign(
        date=pd.to_datetime(crime_scene_report["date"], format="%Y%m%d")
    )
    .set_index("date")
    .loc["1/15/2018"]
    .pipe(lambda df: df.loc[(df["type"] == "murder") & (df["city"] == "SQL City")])
)
display(deposition)


,type,description,city
date,,,
2018-01-15,murder,Security footage shows that there were 2 witne...,SQL City


In [5]:
display(Markdown("## Clue 2"))
display(Markdown(deposition["description"].values[0]))



## Clue 2

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

In [6]:
def namestr(obj, namespace=locals()) -> str:
    return [name for name in namespace if namespace[name] is obj][0]



In [7]:
def print_table_names():
    print([namestr(d) for d in db])



In [8]:
for d in db:
    print(namestr(d))
    display(d.sample())
    print()


drivers_license


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
7900,812448,66,81,amber,white,male,7Y16Q2,Chevrolet,Trailblazer



income


,ssn,annual_income
4099,591595955,74500.0



get_fit_now_members


...

,person_id,event_id,event_name,date
16382,33888,7946,Things are more like they used to be than they...,20180410



get_fit_now_check_in


,membership_id,check_in_date,check_in_time,check_out_time
999,I0UIP,20170424,1023,1145



crime_scene_report


,date,type,description,city
72,20170122,murder,"came near her, she began, in a low, timid voic...",Temecula


In [9]:
witness_1 = person.loc[person["address_street_name"] == "Northwestern Dr"].loc[
    lambda df: df["address_number"] == df["address_number"].max()
]
display(witness_1)


,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [10]:
witness_2 = person.loc[
    (person["name"].str.contains("Annabel"))
    & (person["address_street_name"] == "Franklin Ave")
]
display(witness_2)


,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [11]:
witnesses = pd.concat([witness_1, witness_2])


In [12]:
clue_3 = interview.loc[interview["person_id"].isin(witnesses["id"])].merge(
    witnesses, left_on="person_id", right_on="id"
)


In [13]:
display(Markdown("## Clue 3"))
for t in clue_3["transcript"]:
    display(Markdown(t))
    print()


## Clue 3

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.

In [14]:
suspect = (
    get_fit_now_members.loc[
        (get_fit_now_members["membership_status"] == "gold")
        & (get_fit_now_members["id"].str[:3] == "48Z")
    ]
    .merge(person, left_on="person_id", right_on="id", suffixes=["_gym", "_person"])
    .merge(
        drivers_license.loc[drivers_license["plate_number"].str.contains("H42W")],
        left_on="license_id",
        right_on="id",
        suffixes=["_person", "_driver"],
    )
).merge(
    get_fit_now_check_in.assign(
        check_in_date=pd.to_datetime(
            get_fit_now_check_in["check_in_date"], format="%Y%m%d"
        ),
    )
    .set_index("check_in_date")
    .loc["1/9/2018"],
    left_on="id_gym",
    right_on="membership_id",
)


In [15]:
display(suspect.T)


,0
id_gym,48Z55
person_id,67318
name_gym,Jeremy Bowers
membership_start_date,20160101
membership_status,gold
id_person,67318
name_person,Jeremy Bowers
license_id,423327
address_number,530
address_street_name,"Washington Pl, Apt 3A"


In [16]:
display(Markdown("## Clue 4"))
display(
    Markdown(
        interview.loc[interview["person_id"].isin(suspect["person_id"])][
            "transcript"
        ].iloc[0]
    )
)


## Clue 4

I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.


In [17]:
mastermind_concert = (
    drivers_license.loc[
        (drivers_license["height"].isin(range(65, 68)))
        & (drivers_license["car_make"] == "Tesla")
        & (drivers_license["hair_color"] == "red")
        & (drivers_license["gender"] == "female")
    ]
    .merge(person, left_on="id", right_on="license_id", suffixes=["_driver", "_person"])
    .merge(
        (
            facebook_event_check_in.assign(
                date=pd.to_datetime(facebook_event_check_in["date"], format="%Y%m%d")
            )
            .set_index("date")
            .loc["12/2017"]
            .reset_index()
            .query("event_name == 'SQL Symphony Concert'")
        ),
        left_on="id_person",
        right_on="person_id",
    )
)


In [18]:
display(mastermind_concert)


,id_driver,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_person,name,license_id,address_number,address_street_name,ssn,date,person_id,event_id,event_name
0,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,2017-12-06,99716,1143,SQL Symphony Concert
1,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,2017-12-12,99716,1143,SQL Symphony Concert
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,2017-12-29,99716,1143,SQL Symphony Concert


In [19]:
mastermind = person.loc[
    person["id"].isin(mastermind_concert.drop_duplicates()["person_id"])
]


In [20]:
display(mastermind)


,id,name,license_id,address_number,address_street_name,ssn
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [21]:
query = f"INSERT INTO solution VALUES (1, '{mastermind['name'].iloc[0]}');SELECT value FROM solution;"
print(query)


INSERT INTO solution VALUES (1, 'Miranda Priestly');SELECT value FROM solution;


In [22]:
%%bash
sqlite3 sql-murder-mystery.db "INSERT INTO solution VALUES (1, 'Miranda Priestly');SELECT value FROM solution;"


Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!
